In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 272)}
dim_z = 20
dim_z_supervised=8

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [5]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [7]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [10]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [12]:
model = gs.GenerativeSurrogate(rom, trainingData, dim_z=dim_z, dim_z_supervised=dim_z_supervised)

In [8]:
# model.save()
# loaded_model = gs.GenerativeSurrogate()
# loaded_model.load()

In [9]:
for n in range(model.data.n_supervised_samples):
    print('sample == ', n)
    model.log_lambdac_mean[n].max_iter = 3e4
    model.log_lambdac_mean[n].converge(model, model.data.n_supervised_samples, mode=n)

sample ==  0
loss_lambda_c =  45945471369216.0
loss_lambda_c =  357083152384.0
loss_lambda_c =  341057994752.0
loss_lambda_c =  335339487232.0
loss_lambda_c =  332537856000.0
loss_lambda_c =  331105796096.0
Epoch  2503: reducing learning rate of group 0 to 3.0000e-03.
Epoch  2520: reducing learning rate of group 0 to 3.0000e-04.
Epoch  2536: reducing learning rate of group 0 to 3.0000e-05.
Epoch  2552: reducing learning rate of group 0 to 3.0000e-06.
sample ==  1
loss_lambda_c =  20042003513344.0
loss_lambda_c =  196687839232.0
loss_lambda_c =  190270406656.0
loss_lambda_c =  188073230336.0
Epoch  1718: reducing learning rate of group 0 to 3.0000e-03.
Epoch  1735: reducing learning rate of group 0 to 3.0000e-04.
Epoch  1751: reducing learning rate of group 0 to 3.0000e-05.
Epoch  1767: reducing learning rate of group 0 to 3.0000e-06.
sample ==  2
loss_lambda_c =  655100825567232.0
Epoch    83: reducing learning rate of group 0 to 3.0000e-03.
loss_lambda_c =  2471118766080.0
loss_lambda

In [10]:
model.fit(n_steps=5, with_precisions=False, z_iterations=10, thetac_iterations=10000, thetaf_iterations=10, lambdac_iterations=10)

step =  0
loss z =  12748624.0
loss z =  12723624.0
loss z =  12700334.0
loss z =  12678778.0
loss z =  12658641.0
loss z =  12639934.0
loss z =  12622554.0
loss z =  12606394.0
loss z =  12591324.0
z step =  5.24725079536438 s
loss_f =  12559499.0
loss_f =  12141416.0
thetaf step =  6.934699296951294 s
sample ==  0
loss_lambda_c =  331093868544.0
Epoch  2568: reducing learning rate of group 0 to 1.0000e-03.
Epoch  2584: reducing learning rate of group 0 to 1.0000e-04.
Epoch  2600: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2616: reducing learning rate of group 0 to 1.0000e-06.
sample ==  1
loss_lambda_c =  187719385088.0
Epoch  1783: reducing learning rate of group 0 to 1.0000e-03.
Epoch  1799: reducing learning rate of group 0 to 1.0000e-04.
Epoch  1815: reducing learning rate of group 0 to 1.0000e-05.
Epoch  1831: reducing learning rate of group 0 to 1.0000e-06.
sample ==  2
loss_lambda_c =  1249009074176.0
loss_lambda_c =  1241229295616.0
Epoch  8610: reducing learning

loss_c =  19858.251953125
loss_c =  19556.1015625
loss_c =  19250.921875
loss_c =  18942.8359375
loss_c =  18631.95703125
loss_c =  18318.4140625
loss_c =  18002.341796875
loss_c =  17683.876953125
loss_c =  17363.169921875
loss_c =  17040.376953125
loss_c =  16715.65625
loss_c =  16389.18359375
loss_c =  16061.1259765625
loss_c =  15731.669921875
loss_c =  15401.0029296875
loss_c =  15069.318359375
loss_c =  14736.818359375
loss_c =  14403.705078125
loss_c =  14070.1884765625
loss_c =  13736.482421875
loss_c =  13402.8046875
loss_c =  13069.3779296875
loss_c =  12736.421875
loss_c =  12404.1611328125
loss_c =  12072.8125
loss_c =  11742.59375
loss_c =  11413.697265625
loss_c =  11086.2861328125
loss_c =  10760.421875
loss_c =  10435.8564453125
loss_c =  10110.6826171875
loss_c =  9751.181640625
loss_c =  9406.8818359375
loss_c =  9070.755859375
loss_c =  8738.939453125
loss_c =  8411.693359375
loss_c =  8089.2568359375
loss_c =  7771.7646484375
loss_c =  7459.13134765625
loss_c =  715

loss_c =  219.93344116210938
loss_c =  219.9246368408203
loss_c =  219.91827392578125
loss_c =  219.91372680664062
loss_c =  219.9104461669922
loss_c =  219.90818786621094
loss_c =  219.9066162109375
loss_c =  219.905517578125
loss_c =  219.90475463867188
loss_c =  219.90426635742188
loss_c =  219.90390014648438
loss_c =  219.9036865234375
loss_c =  219.903564453125
loss_c =  219.90345764160156
loss_c =  219.90338134765625
loss_c =  219.90335083007812
loss_c =  219.90335083007812
loss_c =  219.90330505371094
loss_c =  219.90330505371094
loss_c =  219.9033203125
loss_c =  219.90328979492188
loss_c =  219.9033203125
Epoch 12501: reducing learning rate of group 0 to 1.4000e-04.
loss_c =  219.90330505371094
loss_c =  219.90330505371094
loss_c =  219.90328979492188
loss_c =  219.90330505371094
loss_c =  219.9033203125
loss_c =  219.90328979492188
loss_c =  219.90330505371094
loss_c =  219.90328979492188
loss_c =  219.90330505371094
loss_c =  219.90330505371094
loss_c =  219.90328979492188
l

loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.94985961914062
loss_c =  219.94985961914062
loss_c =  219.94985961914062
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.94989013671875
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.9498748779297
Epoch 18507: reducing learning rate of group 0 to 1.4000e-04.
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.9498748779297
loss_c =  219.94989013671875
loss_c =  219.9498901367187

Epoch  2006: reducing learning rate of group 0 to 3.0000e-08.
sample ==  15
loss_lambda_c =  54343307264.0
Epoch  1561: reducing learning rate of group 0 to 3.0000e-05.
Epoch  1577: reducing learning rate of group 0 to 3.0000e-06.
Epoch  1593: reducing learning rate of group 0 to 3.0000e-07.
Epoch  1609: reducing learning rate of group 0 to 3.0000e-08.
lambdac step =  1.1580958366394043 s
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.9640655517578
loss_c =  219.964111328125
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.9640655517578
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.96408081054688
loss_c =  219.9640655517578
loss_c =  219.96405029296875
loss_c =  219.96408081054688
loss_c =  219.9640655517578
Epoch

loss_lambda_c =  257742323712.0
Epoch  4898: reducing learning rate of group 0 to 1.0000e-05.
Epoch  4914: reducing learning rate of group 0 to 1.0000e-06.
Epoch  4930: reducing learning rate of group 0 to 1.0000e-07.
Epoch  4946: reducing learning rate of group 0 to 1.0000e-08.
sample ==  14
loss_lambda_c =  507602567168.0
Epoch  2022: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2038: reducing learning rate of group 0 to 1.0000e-06.
Epoch  2054: reducing learning rate of group 0 to 1.0000e-07.
Epoch  2070: reducing learning rate of group 0 to 1.0000e-08.
sample ==  15
loss_lambda_c =  54343245824.0
Epoch  1625: reducing learning rate of group 0 to 1.0000e-05.
Epoch  1641: reducing learning rate of group 0 to 1.0000e-06.
Epoch  1657: reducing learning rate of group 0 to 1.0000e-07.
Epoch  1673: reducing learning rate of group 0 to 1.0000e-08.
lambdac step =  1.297306776046753 s
loss_c =  219.9687042236328
loss_c =  219.9686279296875
loss_c =  219.96810913085938
loss_c =  21

In [11]:
model.fit(n_steps=10000, z_iterations=50, with_precisions=True, thetaf_iterations=200, thetac_iterations=500, lambdac_iterations=100)

step =  5
loss z =  12102745.0
loss z =  12101985.0
loss z =  12101238.0
loss z =  12100503.0
Epoch    54: reducing learning rate of group 0 to 2.0000e-05.
loss z =  12099777.0
loss z =  12099635.0
loss z =  12099492.0
loss z =  12099349.0
loss z =  12099207.0
loss z =  12099068.0
loss z =  12098927.0
loss z =  12098785.0
loss z =  12098645.0
loss z =  12098506.0
loss z =  12098366.0
Epoch    65: reducing learning rate of group 0 to 4.0000e-06.
loss z =  12098228.0
loss z =  12098199.0
loss z =  12098171.0
loss z =  12098142.0
loss z =  12098087.0
loss z =  12098060.0
loss z =  12098031.0
loss z =  12098003.0
loss z =  12097976.0
loss z =  12097948.0
Epoch    76: reducing learning rate of group 0 to 8.0000e-07.
loss z =  12097920.0
loss z =  12097916.0
loss z =  12097909.0
loss z =  12097904.0
loss z =  12097898.0
loss z =  12097893.0
loss z =  12097888.0
loss z =  12097881.0
loss z =  12097876.0
loss z =  12097870.0
loss z =  12097864.0
Epoch    87: reducing learning rate of group 0 t

loss_f =  9161749.0
loss_f =  9149955.0
loss_f =  9137338.0
loss_f =  9127921.0
loss_f =  9119121.0
loss_f =  9110574.0
loss_f =  9102600.0
loss_f =  9094747.0
loss_f =  9087185.0
loss_f =  9079846.0
loss_f =  9072704.0
loss_f =  9065800.0
loss_f =  9059102.0
loss_f =  9052601.0
loss_f =  9046259.0
loss_f =  9040058.0
loss_f =  9033962.0
loss_f =  9027976.0
loss_f =  9041393.0
loss_f =  9016392.0
loss_f =  9010658.0
loss_f =  9004854.0
loss_f =  8999164.0
loss_f =  8993521.0
loss_f =  8987916.0
loss_f =  8982344.0
loss_f =  8976816.0
loss_f =  8971324.0
loss_f =  8965867.0
loss_f =  8960444.0
loss_f =  8955057.0
loss_f =  8949705.0
loss_f =  8944394.0
loss_f =  8939115.0
thetaf step =  139.2079062461853 s
sample ==  0
loss_lambda_c =  11067.1513671875
Epoch  2953: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2969: reducing learning rate of group 0 to 1.0000e-06.
Epoch  2985: reducing learning rate of group 0 to 1.0000e-07.
Epoch  3001: reducing learning rate of group 0 to 1.

Epoch  9108: reducing learning rate of group 0 to 1.0000e-08.
sample ==  3
loss_lambda_c =  43826.3984375
Epoch  4100: reducing learning rate of group 0 to 1.0000e-05.
Epoch  4116: reducing learning rate of group 0 to 1.0000e-06.
Epoch  4132: reducing learning rate of group 0 to 1.0000e-07.
Epoch  4148: reducing learning rate of group 0 to 1.0000e-08.
sample ==  4
loss_lambda_c =  24291.0078125
Epoch  5162: reducing learning rate of group 0 to 1.0000e-05.
Epoch  5178: reducing learning rate of group 0 to 1.0000e-06.
Epoch  5194: reducing learning rate of group 0 to 1.0000e-07.
Epoch  5210: reducing learning rate of group 0 to 1.0000e-08.
sample ==  5
loss_lambda_c =  37792.8046875
Epoch  2300: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2316: reducing learning rate of group 0 to 1.0000e-06.
Epoch  2332: reducing learning rate of group 0 to 1.0000e-07.
Epoch  2348: reducing learning rate of group 0 to 1.0000e-08.
sample ==  6
loss_lambda_c =  23237.1640625
Epoch  3187: reduc

Epoch  3703: reducing learning rate of group 0 to 1.0000e-06.
Epoch  3719: reducing learning rate of group 0 to 1.0000e-07.
Epoch  3735: reducing learning rate of group 0 to 1.0000e-08.
sample ==  9
loss_lambda_c =  8165.09423828125
Epoch  2198: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2214: reducing learning rate of group 0 to 1.0000e-06.
Epoch  2230: reducing learning rate of group 0 to 1.0000e-07.
Epoch  2246: reducing learning rate of group 0 to 1.0000e-08.
sample ==  10
loss_lambda_c =  19093.2734375
Epoch  3734: reducing learning rate of group 0 to 1.0000e-05.
Epoch  3750: reducing learning rate of group 0 to 1.0000e-06.
Epoch  3766: reducing learning rate of group 0 to 1.0000e-07.
Epoch  3782: reducing learning rate of group 0 to 1.0000e-08.
sample ==  11
loss_lambda_c =  10354.796875
Epoch  8719: reducing learning rate of group 0 to 1.0000e-05.
Epoch  8735: reducing learning rate of group 0 to 1.0000e-06.
Epoch  8751: reducing learning rate of group 0 to 1.0000e-

Epoch  5270: reducing learning rate of group 0 to 1.0000e-08.
sample ==  14
loss_lambda_c =  15625.32421875
Epoch  2343: reducing learning rate of group 0 to 1.0000e-05.
Epoch  2359: reducing learning rate of group 0 to 1.0000e-06.
Epoch  2375: reducing learning rate of group 0 to 1.0000e-07.
Epoch  2391: reducing learning rate of group 0 to 1.0000e-08.
sample ==  15
loss_lambda_c =  1820.1455078125
Epoch  1947: reducing learning rate of group 0 to 1.0000e-05.
Epoch  1963: reducing learning rate of group 0 to 1.0000e-06.
Epoch  1979: reducing learning rate of group 0 to 1.0000e-07.
Epoch  1995: reducing learning rate of group 0 to 1.0000e-08.
lambdac step =  1.1771996021270752 s
loss_c =  263.59075927734375
loss_c =  258.0021057128906
loss_c =  256.96026611328125
loss_c =  256.08941650390625
loss_c =  255.296875
loss_c =  254.5257568359375
loss_c =  253.7234344482422
loss_c =  252.83477783203125
loss_c =  251.8072509765625
loss_c =  250.62158203125
thetac step =  2.325016736984253 s
st

...

Epoch  7948: reducing learning rate of group 0 to 1.0000e-07.
Epoch  7964: reducing learning rate of group 0 to 1.0000e-08.
sample ==  9
loss_lambda_c =  8139.42578125
Epoch  6425: reducing learning rate of group 0 to 1.0000e-05.
Epoch  6441: reducing learning rate of group 0 to 1.0000e-06.
Epoch  6457: reducing learning rate of group 0 to 1.0000e-07.
Epoch  6473: reducing learning rate of group 0 to 1.0000e-08.
sample ==  10
loss_lambda_c =  19108.955078125
Epoch  7958: reducing learning rate of group 0 to 1.0000e-05.
Epoch  7974: reducing learning rate of group 0 to 1.0000e-06.
Epoch  7990: reducing learning rate of group 0 to 1.0000e-07.
Epoch  8006: reducing learning rate of group 0 to 1.0000e-08.
sample ==  11
loss_lambda_c =  10356.9013671875
Epoch 12946: reducing learning rate of group 0 to 1.0000e-05.
Epoch 12962: reducing learning rate of group 0 to 1.0000e-06.
Epoch 12978: reducing learning rate of group 0 to 1.0000e-07.
Epoch 12994: reducing learning rate of group 0 to 1.000

Epoch  9498: reducing learning rate of group 0 to 1.0000e-08.
sample ==  14
loss_lambda_c =  15623.056640625
Epoch  6567: reducing learning rate of group 0 to 1.0000e-05.
Epoch  6583: reducing learning rate of group 0 to 1.0000e-06.
Epoch  6599: reducing learning rate of group 0 to 1.0000e-07.
Epoch  6615: reducing learning rate of group 0 to 1.0000e-08.
sample ==  15
loss_lambda_c =  1814.5977783203125
Epoch  6173: reducing learning rate of group 0 to 1.0000e-05.
Epoch  6189: reducing learning rate of group 0 to 1.0000e-06.
Epoch  6205: reducing learning rate of group 0 to 1.0000e-07.
Epoch  6221: reducing learning rate of group 0 to 1.0000e-08.
lambdac step =  1.185487985610962 s
loss_c =  256.26202392578125
Epoch 74548: reducing learning rate of group 0 to 2.0000e-05.
loss_c =  255.60238647460938
loss_c =  255.54898071289062
loss_c =  255.4976806640625
loss_c =  255.44723510742188
loss_c =  255.3970184326172
loss_c =  255.34661865234375
loss_c =  255.2956085205078
loss_c =  255.2439

loss z =  9716994.0
loss z =  9708488.0
loss z =  9703434.0
loss z =  9699978.0
loss z =  9697159.0
loss z =  9694627.0
loss z =  9692255.0
loss z =  9689993.0
loss z =  9687815.0
loss z =  9685710.0
Epoch  3270: reducing learning rate of group 0 to 2.0000e-05.
loss z =  9683664.0
loss z =  9683263.0
loss z =  9682864.0
loss z =  9682467.0
loss z =  9682075.0
loss z =  9681683.0
loss z =  9681293.0
loss z =  9680905.0
loss z =  9680132.0
loss z =  9679750.0
Epoch  3281: reducing learning rate of group 0 to 4.0000e-06.
loss z =  9679371.0
loss z =  9679297.0
loss z =  9679219.0
loss z =  9679143.0
loss z =  9679068.0
loss z =  9678992.0
loss z =  9678917.0
loss z =  9678842.0
loss z =  9678766.0
loss z =  9678688.0
loss z =  9678614.0
Epoch  3292: reducing learning rate of group 0 to 8.0000e-07.
loss z =  9678538.0
loss z =  9678522.0
loss z =  9678508.0
loss z =  9678492.0
loss z =  9678477.0
loss z =  9678462.0
loss z =  9678431.0
loss z =  9678416.0
loss z =  9678402.0
loss z =  9678

loss z =  9663558.0
loss z =  9663542.0
loss z =  9663526.0
loss z =  9663510.0
Epoch  3348: reducing learning rate of group 0 to 1.6000e-07.
z step =  25.912616968154907 s
loss_f =  7073211.0
loss_f =  7066216.5
Epoch 14583: reducing learning rate of group 0 to 2.5000e-04.
loss_f =  7063882.0
loss_f =  7060578.5
loss_f =  7057621.5
loss_f =  7055419.5
loss_f =  7053732.0
loss_f =  7052319.5
Epoch 14614: reducing learning rate of group 0 to 1.2500e-04.
loss_f =  7051157.0
loss_f =  7050643.0
loss_f =  7050175.0
loss_f =  7049719.0
loss_f =  7049312.0
loss_f =  7048926.0
loss_f =  7048550.0
loss_f =  7048187.5
loss_f =  7047838.5
loss_f =  7047500.0
loss_f =  7047159.5
loss_f =  7046842.0
loss_f =  7046522.0
loss_f =  7046197.0
loss_f =  7045893.0
loss_f =  7045583.0
loss_f =  7045274.5
loss_f =  7044974.0
loss_f =  7044678.0
loss_f =  7044374.0
loss_f =  7044084.0
loss_f =  7043793.5
loss_f =  7043509.0
loss_f =  7043218.5
loss_f =  7042942.5
loss_f =  7042643.0
loss_f =  7042362.0
los

KeyboardInterrupt: 

In [ ]:
model.tauc

In [ ]:
model.z_mean

In [ ]:
# model = gs.GenerativeSurrogate()
# model.load()

In [ ]:
test_samples = {n for n in range(0, 4)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z, grad_norm = model.predict(testData, max_iterations=3000, optimizer='SGD', lr=1e-3)

In [ ]:
fig = plt.figure()
plt.plot(grad_norm)
plt.yscale('log')
plt.grid()
ax = plt.gca()
ax.set_ylim(1e1, 1e5)

In [ ]:
plt.xlabel('iteration')
plt.ylabel('gradient norm')
plt.title('Gradient norm w.r.t. z in prediction inference')

In [ ]:
grad_norm[-10:]

In [ ]:
model.z_mean[:4, :].data

In [ ]:
Z.data

In [ ]:
uf_pred

In [ ]:
model.data.P[:4, :]

In [ ]:
model.fit(n_steps=10)

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred[3], (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[3, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
lg_lambdac = model.pcNet(Z)

In [ ]:
lg_lambdac

In [ ]:
lg_lambdac2 = model.pcNet(model.z_mean[:4, :])
lg_lambdac2

In [ ]:
model.log_lambdac_mean_tensor[:4, :]

In [ ]:
uf_pred2 = model.rom_autograd(torch.exp(lg_lambdac2[0]))

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred2.detach().numpy(), (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
model.log_lambdac_mean_tensor[:4, :]

In [ ]:
fig = plt.figure()
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

ax = []
for i in range(4):
    ax.append(fig.add_subplot(2, 4, i + 1, projection='3d'))
    ax[i].plot_surface(x, y, np.reshape(uf_pred[i], (129, 129)), edgecolor='none')
    ax[i].plot_surface(x, y, np.reshape(model.data.P[i, :], (129, 129)).detach().numpy(),
                       edgecolor='none', cmap='inferno')
    ax[i].set_position([0.025 + .2375*i, 0.55, 0.22, 0.4])
    ax[i].elev = 10
    ax[i].set_title('pressure field prediction')
    
xx = np.array([i for i in range(dim_z)])
width = 0.42  # the width of the bars
for i in range(4):
    ax.append(fig.add_subplot(2, 4, i + 5))
    ax[i + 4].bar(xx-width/2, Z[i, :].detach().numpy(), width, label='prediction')
    ax[i + 4].bar(xx + width/2, model.z_mean[i, :].detach().numpy(), width, label='training')
    ax[i + 4].set_position([0.05 + .235*i, 0.08, 0.20, 0.4])
    ax[i + 4].set_title('location of z distribution')
    ax[i + 4].set_xlabel('component')
    ax[i + 4].set_ylabel('z')
    ax[i + 4].legend()

In [ ]:
lg_lambdac2 = model.pcNet(model.z_mean[:4, :])
# lg_lambdac2 = model.log_lambdac_mean_tensor[:4, :]
uf_pred2 = []
fig = plt.figure()
figManager = plt.get_current_fig_manager()
figManager.window.showMaximized()

x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose
ax = []
for i in range(4):
    uf_pred2.append(model.rom_autograd(torch.exp(lg_lambdac2[i])))
    ax.append(fig.add_subplot(1, 4, i + 1, projection='3d'))
    ax[i].plot_surface(x, y, np.reshape(uf_pred2[i].detach().numpy(), (129, 129)), edgecolor='none')
    ax[i].plot_surface(x, y, np.reshape(model.data.P[i, :], (129, 129)).detach().numpy(),
                       edgecolor='none', cmap='inferno')
    ax[i].set_position([0.025 + .2375*i, 0.55, 0.22, 0.4])
    ax[i].elev = 10
    ax[i].set_title('pressure field prediction')

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection='3d')
ax.plot_surface(x, y, np.reshape(uf_pred2[i].detach().numpy(), (129, 129)), edgecolor='none')

In [ ]:
plt.close('all')

In [ ]:
ax[0].get_position()

In [ ]:
ax[0].set_position([0.02, 0.3, 0.2, 0.4])

In [ ]:
ax[0].elev = 10

In [ ]:
ax[0].get_zticks()

In [ ]:
ax[5].set_xlabel

In [ ]:
a

In [ ]:
fc = torch.nn.Linear(5, 2)

In [ ]:
fc.weight.shape

In [ ]:
fc.bias

In [ ]:
model.training_iterations = 0

In [ ]:
from torch.distributions import constraints
from torch.distributions.exp_family import ExponentialFamily
from torch.distributions.utils import _standard_normal, broadcast_all

In [ ]:
dist = torch.distributions.Normal(0, 1)

In [ ]:
dist.stddev = 2

In [ ]:
x = torch.randn(3, 4)

In [ ]:
x

In [ ]:
sum(x)

In [ ]:
def log_emp_dist(x):
    sigma_e = 2*torch.ones(1)
    mu_e = -3*torch.ones(1)
    return .5*(1/sigma_e**2)*(x - mu_e)**2

def log_emp_dist_grad(x):
    sigma_e = 2*torch.ones(1)
    mu_e = -3*torch.ones(1)
    return (x - mu_e)/sigma_e**2
    

In [ ]:
from VI import variationalinference as vi
svi = vi.DiagGaussianSVI(log_emp_dist, log_emp_dist_grad, 1)

In [ ]:
svi.fit()

In [ ]:
for i in range(10000):
    loss = elbo.autograd_elbo(elbo.vi_mean, elbo.vi_log_std)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

In [ ]:
print('sigma == ', elbo.vi_std)
print('mu == ', elbo.vi_mean)

In [ ]:
svi.vi_std

In [ ]:
log_sigma.grad

In [ ]:
torch.zeros(5) - 1

In [13]:
d = {}

In [17]:
d['mean'] = 5

In [18]:
d

{'mean': 5}

In [58]:
dist = torch.distributions.multivariate_normal.MultivariateNormal(torch.zeros(3, 4, requires_grad=True), covariance_matrix=torch.eye(4).unsqueeze(0).repeat(3, 1, 1))

In [37]:
a = torch.tensor([1, 2, 3])

In [42]:
a = a.unsqueeze(0)

In [44]:
a = torch.ones(3, requires_grad=True)
b = torch.ones(4, requires_grad=True)

In [45]:
opt = optim.Adam([a, b])

In [6]:
dist

MultivariateNormal(loc: torch.Size([3]), covariance_matrix: torch.Size([3, 3]))

In [8]:
dist.expand([3])

MultivariateNormal(loc: torch.Size([3, 3]), covariance_matrix: torch.Size([3, 3, 3]))

In [12]:
dist_e = dist.expand([4])

In [15]:
dist_e.covariance_matrix

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]],

        [[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.]]])

In [40]:
L = torch.eye(2)
L[0, 1] = .5

In [41]:
L = L.unsqueeze(0).repeat(3, 1, 1)

In [44]:
L @ torch.transpose(L, 1, 2)

tensor([[[1.2500, 0.5000],
         [0.5000, 1.0000]],

        [[1.2500, 0.5000],
         [0.5000, 1.0000]],

        [[1.2500, 0.5000],
         [0.5000, 1.0000]]])

In [56]:
eps = torch.randn(3,2)
eps

tensor([[ 0.7497,  0.3456],
        [ 1.9351,  1.3505],
        [ 1.1620, -0.4637]])

In [57]:
torch.bmm(L, eps.unsqueeze(2))

tensor([[[ 0.9225],
         [ 0.3456]],

        [[ 2.6103],
         [ 1.3505]],

        [[ 0.9302],
         [-0.4637]]])

In [54]:
L.shape

torch.Size([3, 2, 2])

In [55]:
eps.unsqueeze(2).shape

torch.Size([2, 2, 1])